In [1]:
import os
from pyspark.sql import SparkSession

In [2]:
# Konfigurasi diambil dari environment variables yang di-set di docker-compose
aws_access_key_id = os.environ.get("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.environ.get("AWS_SECRET_ACCESS_KEY")
catalog_s3_endpoint = os.environ.get("CATALOG_S3_ENDPOINT")

# Membuat SparkSession
spark = SparkSession.builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", catalog_s3_endpoint) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", aws_access_key_id) \
    .config("spark.hadoop.fs.s3a.secret.key", aws_secret_access_key) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

print("SparkSession berhasil dibuat dengan paket S3!")

SparkSession berhasil dibuat dengan paket S3!


25/08/01 07:10:11 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
spark._jsc.hadoopConfiguration().get("fs.s3a.impl")

'org.apache.hadoop.fs.s3a.S3AFileSystem'

In [4]:
print(spark)

In [5]:
active_session = SparkSession.getActiveSession()

if active_session:
    print("✅ SparkSession sudah aktif.")
    print(active_session)
else:
    print("❌ SparkSession belum terbentuk.")

✅ SparkSession sudah aktif.


In [6]:
# Mencoba menjalankan query sederhana
spark.sql("SELECT 'Spark Session Aktif!' as status").show()

+--------------------+
|              status|
+--------------------+
|Spark Session Aktif!|
+--------------------+



In [7]:
%%sql

SHOW CATALOGS 

25/08/01 07:42:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


catalog
minilake
spark_catalog


In [9]:
%%sql

SHOW DATABASES

namespace
sample


In [13]:
# Ganti 'coffeeshop' dengan nama bucket yang ada di MinIO Anda
# Jika bucket kosong, ini tidak akan error.
# Jika bucket tidak ada atau kredensial salah, ini akan error.

try:
    spark.read.load("s3a://coffeeshop/")
    print("✅ Konfigurasi Benar! Spark berhasil menghubungi MinIO.")
except Exception as e:
    print(f"❌ Gagal terhubung atau mengakses bucket di MinIO. Error: {e}")

❌ Gagal terhubung atau mengakses bucket di MinIO. Error: An error occurred while calling o68.load.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.

25/07/31 17:53:07 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://coffeeshop/.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSour

In [3]:
# Tentukan path ke file di dalam MinIO
bucket_name = "coffeeshop"
file_name = "transaksi.csv"
file_path = f"s3a://{bucket_name}/{file_name}"

# Baca file CSV ke dalam DataFrame Spark
# header=True -> Menganggap baris pertama sebagai nama kolom
# inferSchema=True -> Spark akan mencoba menebak tipe data setiap kolom
df_penjualan = spark.read.csv(file_path, header=True, inferSchema=True)

# Tampilkan isi DataFrame dan skemanya
print(f"Berhasil membaca file dari: {file_path}")
df_penjualan.show()
df_penjualan.printSchema()

25/07/31 16:57:53 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://coffeeshop/transaksi.csv.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRela

Py4JJavaError: An error occurred while calling o50.csv.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(Dat